In [1]:
!pip install sacrebleu
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 5.0 MB/s eta 0:00:00a 0:00:01


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.utils.data import Dataset, DataLoader
import datasets
from transformers import AdamW
import torch.nn as nn
from torch.cuda.amp import GradScaler, autocast

import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [3]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM

config = PeftConfig.from_pretrained("KapitalK/mGPT")
base_model = AutoModelForCausalLM.from_pretrained("ai-forever/mGPT")
model = PeftModel.from_pretrained(base_model, "KapitalK/mGPT").to('cuda')
tokenizer = AutoTokenizer.from_pretrained("ai-forever/mGPT")

# model_name = "ai-forever/mGPT"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name).to('cuda')

adapter_config.json:   0%|          | 0.00/495 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/738 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.45G [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/66.8k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:488: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  adapters_weights = torch.load(filename, map_location=t

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.89M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/606 [00:00<?, ?B/s]

In [4]:
def preprocess_function(examples):
    inputs = tokenizer(examples['source'], max_length=128, truncation=True, padding='max_length')
    targets = tokenizer(examples['target'], max_length=128, truncation=True, padding='max_length')
    inputs['labels'] = targets['input_ids']
    return inputs

dataset = pd.read_csv("/kaggle/input/mansi-russian-parralel-corpus/overall_80K.csv", index_col=0)

In [5]:
texts = dataset['source'].tolist() + dataset['target'].tolist()
unique_tokens = set()
for text in tqdm(texts):
    tokens = tokenizer.tokenize(text)
    unique_tokens.update(tokens)
    
num_added_tokens = tokenizer.add_tokens(list(unique_tokens))

print(f"Added {num_added_tokens} new tokens.")

100%|██████████| 162292/162292 [00:22<00:00, 7233.89it/s]


Added 10584 new tokens.


In [6]:
model.resize_token_embeddings(len(tokenizer))

Embedding(100000, 2048)

In [7]:
train_df, test_val_df = train_test_split(dataset, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(test_val_df, test_size=0.5, random_state=42)

train_dataset = datasets.Dataset.from_pandas(train_df)
val_dataset = datasets.Dataset.from_pandas(val_df)
test_dataset = datasets.Dataset.from_pandas(test_df)

train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/64916 [00:00<?, ? examples/s]

Map:   0%|          | 0/8115 [00:00<?, ? examples/s]

Map:   0%|          | 0/8115 [00:00<?, ? examples/s]

In [8]:
class CustomDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        return {
            'input_ids': torch.tensor(item['input_ids']),
            'attention_mask': torch.tensor(item['attention_mask']),
            'labels': torch.tensor(item['labels'])
        }

train_loader = DataLoader(CustomDataset(train_dataset), batch_size=8, shuffle=True)
eval_loader = DataLoader(CustomDataset(test_dataset), batch_size=8)

In [9]:
from torch.utils.data import DataLoader
from tqdm import tqdm

def train(model, train_loader, optimizer, criterion, scaler):
    model.train()
    epoch_loss = 0

    for i, batch in tqdm(enumerate(train_loader), total=len(train_loader), position=0, leave=True):
        input_ids = batch['input_ids'].to('cuda')
        attention_mask = batch['attention_mask'].to('cuda')
        labels = batch['labels'].to('cuda')

        optimizer.zero_grad()

        with autocast():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        epoch_loss += loss.item()
        
        del input_ids, attention_mask, labels
        torch.cuda.empty_cache()
        
        if i%100==0:
            print(f'Step {i + 1}, Train Loss: {epoch_loss / (i+1):.4f}')

    return epoch_loss / len(train_loader)

In [10]:
optimizer = AdamW(model.parameters(), lr=5e-5)
scaler = GradScaler()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/tmp/ipykernel_36/2643206744.py:2: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [11]:
n_epochs = 1

for epoch in tqdm(range(n_epochs)):
    train_loss = train(model, train_loader, optimizer, nn.CrossEntropyLoss(), scaler)

  0%|          | 0/8115 [00:00<?, ?it/s]/tmp/ipykernel_36/1000605197.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
  0%|          | 1/8115 [00:01<4:14:53,  1.88s/it]

Step 1, Train Loss: 26.2023


  1%|          | 101/8115 [02:21<3:06:06,  1.39s/it]

Step 101, Train Loss: 23.5420


  2%|▏         | 201/8115 [04:40<3:03:49,  1.39s/it]

...

Step 401, Train Loss: 20.6402


  6%|▌         | 501/8115 [11:38<2:56:54,  1.39s/it]

Step 501, Train Loss: 20.0456


  0%|          | 0/1 [12:26<?, ?it/s]


KeyboardInterrupt: 

In [12]:
import random
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

random.seed(42) 
# random_indices = random.sample(range(len(test_dataset)), 1000)
random_test_samples = test_dataset

def generate_translation(sample):
    input_ids = torch.tensor(sample['input_ids']).unsqueeze(0).to(device)  # Конвертируем список в тензор
    with torch.no_grad():
        generated_ids = model.generate(input_ids, max_length=200, num_beams=5, early_stopping=True)
    return tokenizer.decode(generated_ids[0], skip_special_tokens=True)

original_texts = []
correct_translations = []
model_translations = []

for i, sample in tqdm(enumerate(random_test_samples)):
    input_text = tokenizer.decode(sample['input_ids'], skip_special_tokens=True)
    correct_translation = tokenizer.decode(sample['labels'], skip_special_tokens=True)
    model_translation = generate_translation(sample)

    original_texts.append(input_text)
    correct_translations.append(correct_translation)
    model_translations.append(model_translation)
    break
    
df_results = pd.DataFrame({
    "Original Text": original_texts,
    "Correct Translation": correct_translations,
    "Model Translation": model_translations
})

0it [00:00, ?it/s]/opt/conda/lib/python3.10/site-packages/peft/peft_model.py:1685: UserWarning: Position ids are not supported for parameter efficient tuning. Ignoring position ids.
  warnings.warn("Position ids are not supported for parameter efficient tuning. Ignoring position ids.")
0it [00:16, ?it/s]


In [13]:
df_results

,Original Text,Correct Translation,Model Translation
0,Пётр всего полведра воды принёс.,Петыр вēтра суп вит туп та тотас.,ლებშილებშილებშილებშილებშილებშილებშილებშილებშილ...
